In [1]:
import utils.conf as conf
import preprocessing.data_cleaner as cleaner
import preprocessing.feature_extractor as extractor
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

In [49]:
train, test = cleaner.clean()


In [39]:
train = extractor.aggregate_features_v2(train)


In [20]:
from sklearn.model_selection import train_test_split
y = train['TripType']
X = train.drop('TripType', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, n_jobs=4)
clf.fit(X_train,y_train)
clf_probs = clf.predict_proba(X_test)


importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d : %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]],importances[indices[f]]))
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(clf, X_test, y_test.values)
scores = clf.score(X_test,y_test)
print(scores)

from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, clf_probs)
print(logloss_score)

Feature ranking:
1. feature 3 : FinelineNumber_has_max (0.143909)
2. feature 0 : ScanCountSum (0.086384)
3. feature 5 : Purchased_num (0.073598)
4. feature 1 : Encoded_UPC_nunique (0.060439)
5. feature 23 : DSD GROCERY (0.036789)
6. feature 26 : FINANCIAL SERVICES (0.031983)
7. feature 63 : PRODUCE (0.027422)
8. feature 56 : PERSONAL CARE (0.027124)
9. feature 58 : PHARMACY OTC (0.026834)
10. feature 30 : GROCERY DRY GOODS (0.026183)
11. feature 4 : Returned_num (0.025144)
12. feature 22 : DAIRY (0.022426)
13. feature 66 : SERVICE DELI (0.021750)
14. feature 38 : IMPULSE MERCHANDISE (0.019673)
15. feature 49 : MENS WEAR (0.018560)
16. feature 2 : FinelineNumber_max_num (0.018249)
17. feature 36 : HOUSEHOLD CHEMICALS/SUPP (0.016971)
18. feature 40 : INFANT CONSUMABLE HARDLINES (0.014611)
19. feature 11 : BEAUTY (0.013655)
20. feature 27 : FROZEN FOODS (0.012693)
21. feature 37 : HOUSEHOLD PAPER GOODS (0.012370)
22. feature 59 : PHARMACY RX (0.012093)
23. feature 17 : CANDY, TOBACCO, COO

**XGBoost**

In [23]:
import models.models as models
xgb_clf = models.XGBoostMutliClass(nthread=4, eta=0.1, max_depth=20, num_round=50, silent=1)
xgb_clf.fit(X_train, y_train)
yprob, preds = xgb_clf.predict_proba(X_test)
from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, yprob)
print(logloss_score)

0.863826128917


**Submission**

In [40]:
test = extractor.aggregate_features_v2(test)
y_train = train['TripType']
for col_name in train.columns:
    if col_name not in test.columns:
        train.drop(col_name, axis=1, inplace=True)


In [41]:
import models.models as models
xgb_clf = models.XGBoostMutliClass(nthread=4, eta=0.1, max_depth=20, num_round=50, silent=1)
X_train = train
xgb_clf.fit(X_train, y_train)


XGBoostMutliClass(eta=0.1, max_depth=20, nthread=4, num_round=50, silent=1)

In [76]:
yprob, classes = xgb_clf.predict_proba(test)
column_names = ['VisitNumber']
for cls in classes:
    column_names.append('TripType_' + str(cls))
out = pd.DataFrame(columns=column_names)
out['VisitNumber'] = test.index.values
out.ix[:,'TripType_3':] = yprob
out.to_csv('submission.csv', sep=',', index=False)

In [75]:

out

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.000908,0.000898,0.000898,0.000920,0.002393,0.006728,0.020845,0.000923,0.000900,...,0.000910,0.000901,0.062158,0.066700,0.000911,0.005914,0.003453,0.001896,0.001282,0.006912
1,2,0.002202,0.002124,0.002124,0.002183,0.014363,0.013705,0.006284,0.003511,0.002129,...,0.002153,0.002130,0.031233,0.037344,0.002145,0.002651,0.013171,0.005621,0.002209,0.039025
2,3,0.000273,0.000270,0.000270,0.000277,0.000363,0.000513,0.000347,0.000276,0.000271,...,0.000274,0.000271,0.000274,0.000275,0.000273,0.000274,0.000273,0.000273,0.000274,0.989530
3,4,0.000923,0.000913,0.000913,0.000936,0.002106,0.028287,0.899576,0.000932,0.000915,...,0.000925,0.000915,0.000926,0.000929,0.000922,0.000928,0.000923,0.000923,0.000926,0.020149
4,6,0.000273,0.000270,0.000270,0.000277,0.000293,0.000539,0.000347,0.000276,0.000271,...,0.000274,0.000271,0.000274,0.000275,0.000273,0.000274,0.000273,0.000273,0.000274,0.989573
5,13,0.002091,0.002069,0.002069,0.002120,0.249516,0.563614,0.092431,0.006495,0.002074,...,0.002097,0.002611,0.002105,0.002104,0.002089,0.002103,0.002090,0.002091,0.002098,0.009107
6,14,0.001841,0.001775,0.001775,0.001825,0.010330,0.001783,0.001805,0.001814,0.001780,...,0.001799,0.001780,0.002561,0.750180,0.003176,0.003875,0.041571,0.017453,0.048896,0.001977
7,16,0.000926,0.000916,0.000916,0.001384,0.003010,0.000920,0.000932,0.000936,0.000918,...,0.000929,0.001882,0.001205,0.082017,0.001978,0.001100,0.003681,0.009119,0.002299,0.001149
8,18,0.003373,0.003336,0.003336,0.003579,0.028414,0.003351,0.004146,0.003409,0.003344,...,0.003381,0.017288,0.029007,0.558714,0.005926,0.003739,0.004995,0.007182,0.004210,0.006021
9,21,0.002762,0.046353,0.233837,0.002829,0.002828,0.002675,0.002734,0.002722,0.002670,...,0.015405,0.002671,0.003192,0.441096,0.002951,0.007841,0.009423,0.017049,0.006304,0.009557


In [51]:
test = extractor.aggregate_features_v2(test)


In [67]:
test.shape

(95674, 73)

In [66]:
out

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.000908,0.000898,0.000898,0.000920,0.002393,0.006728,0.020845,0.000923,0.000900,...,0.000910,0.000901,0.062158,0.066700,0.000911,0.005914,0.003453,0.001896,0.001282,0.006912
1,2,0.002202,0.002124,0.002124,0.002183,0.014363,0.013705,0.006284,0.003511,0.002129,...,0.002153,0.002130,0.031233,0.037344,0.002145,0.002651,0.013171,0.005621,0.002209,0.039025
2,3,0.000273,0.000270,0.000270,0.000277,0.000363,0.000513,0.000347,0.000276,0.000271,...,0.000274,0.000271,0.000274,0.000275,0.000273,0.000274,0.000273,0.000273,0.000274,0.989530
3,4,0.000923,0.000913,0.000913,0.000936,0.002106,0.028287,0.899576,0.000932,0.000915,...,0.000925,0.000915,0.000926,0.000929,0.000922,0.000928,0.000923,0.000923,0.000926,0.020149
4,6,0.000273,0.000270,0.000270,0.000277,0.000293,0.000539,0.000347,0.000276,0.000271,...,0.000274,0.000271,0.000274,0.000275,0.000273,0.000274,0.000273,0.000273,0.000274,0.989573
5,13,0.002091,0.002069,0.002069,0.002120,0.249516,0.563614,0.092431,0.006495,0.002074,...,0.002097,0.002611,0.002105,0.002104,0.002089,0.002103,0.002090,0.002091,0.002098,0.009107
6,14,0.001841,0.001775,0.001775,0.001825,0.010330,0.001783,0.001805,0.001814,0.001780,...,0.001799,0.001780,0.002561,0.750180,0.003176,0.003875,0.041571,0.017453,0.048896,0.001977
7,16,0.000926,0.000916,0.000916,0.001384,0.003010,0.000920,0.000932,0.000936,0.000918,...,0.000929,0.001882,0.001205,0.082017,0.001978,0.001100,0.003681,0.009119,0.002299,0.001149
8,18,0.003373,0.003336,0.003336,0.003579,0.028414,0.003351,0.004146,0.003409,0.003344,...,0.003381,0.017288,0.029007,0.558714,0.005926,0.003739,0.004995,0.007182,0.004210,0.006021
9,21,0.002762,0.046353,0.233837,0.002829,0.002828,0.002675,0.002734,0.002722,0.002670,...,0.015405,0.002671,0.003192,0.441096,0.002951,0.007841,0.009423,0.017049,0.006304,0.009557
